In [25]:
%matplotlib inline

import os

import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
import pylab as plt

#FTRAIN = '/home/ec2-user/W207FinalProject/Project/Data/testAugBig2.csv'
FTRAIN = '/home/ec2-user/W207FinalProject/Project/Data/training.csv'
FTEST = '/home/ec2-user/W207FinalProject/Project/Data/test.csv'


def load(test=False, cols=None):
    """Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Pass a list of *cols* if you're only interested in a subset of the
    target columns.
    """
    fname = FTEST if test else FTRAIN
    #df = read_csv(os.path.expanduser(fname))  # load pandas dataframe
    df = read_csv(fname)  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]

    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = y.astype(np.float32)
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        
    else:
        y = None

    return X, y


X, y = load()

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

In [13]:
from sklearn.decomposition import PCA

p = PCA(n_components=500)
bork = p.fit_transform(X)


In [55]:
#p.score_samples()
bork[0].shape
#p.components_.shape
#plt.imshow(p.components_[5].reshape(96,96),cmap='gray')
cX = [toodle[0:256].reshape(16,16) for toodle in bork]
cy = y
#print cX

In [60]:
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
import theano

def float32(k):
    return np.cast['float32'](k)

class AdjustVariable(object):
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)
        
net = NeuralNet(
      layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('conv3', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),
        ('hidden4', layers.DenseLayer),
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    
    input_shape=(None, 1, 16,16),
    conv1_num_filters=8, conv1_filter_size=(2,2), pool1_pool_size=(2,2),
    conv2_num_filters=8, conv2_filter_size=(2,2),  pool2_pool_size=(2,2),
    conv3_num_filters=16, conv3_filter_size=(1,1),  pool3_pool_size=(2,2),
    hidden4_num_units=50, hidden5_num_units=50, 
    output_num_units=30, output_nonlinearity=None,
    
    update_learning_rate=theano.shared(float32(0.1)),
    update_momentum=theano.shared(float32(0.9)),
    
    regression=True,
    #batch_iterator_train = WonkyBatchIterator(batch_size=128),
    
    on_epoch_finished=[
    AdjustVariable('update_learning_rate', start=0.1, stop=0.0001),
    AdjustVariable('update_momentum', start=0.9, stop=0.999),],
    
    #max_epochs=500,
    max_epochs=5,
    verbose=1,
    )

#X, y = load2d()  # load 2-d data

net.fit(cX, cy)

# Training for 1000 epochs will take a while.  We'll pickle the
# trained model so that we can load it back later:
#import cPickle as pickle
#import datetime as dt
#now = str(dt.datetime.now()).replace(' ','_')
#now = str(dt.datetime.now()).replace(' ','_').split('.')[0]
#with open('CNN' + now + '.pickle', 'wb') as f:
#    pickle.dump(net4, f, -1)

TypeError: only integer arrays with one element can be converted to an index

In [45]:
16**2

256